In [2]:
import os
import sagemaker
from sagemaker.tensorflow import TensorFlow


sagemaker_session = sagemaker.Session()

bucket = 'calvinandpogs-ee148'

role = sagemaker.get_execution_role()
print(bucket, role)
print(os.getcwd())

calvinandpogs-ee148 arn:aws:iam::652516965730:role/service-role/AmazonSageMaker-ExecutionRole-20210513T011299
/home/ec2-user/SageMaker/atrw/models/yolov5


#### Full Training Set

In [3]:
estimator = TensorFlow(entry_point='train.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 640,
                            'batch-size': 16,
                            'epochs': 400,
                            'data': 'atrw.yaml',
                            'weights': 'weights/yolov5m.pt',
                            'output-s3': "s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train_m"
                        }
)

## Training Subsets

#### 1/2 of Training Set

In [13]:
estimator = TensorFlow(entry_point='train.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 640,
                            'batch-size': 16,
                            'epochs': 100,
                            'data': 'atrw2.yaml',
                            'weights': 'weights/yolov5s.pt',
                            'output-s3': "s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train2"
                        }
)

#### 1/4 of Training Set

In [11]:
estimator = TensorFlow(entry_point='train.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 640,
                            'batch-size': 16,
                            'epochs': 100,
                            'data': 'atrw4.yaml',
                            'weights': 'weights/yolov5s.pt',
                            'output-s3': "s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train4"
                        }
)

#### 1/8 of Training Set

In [9]:
estimator = TensorFlow(entry_point='train.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 640,
                            'batch-size': 16,
                            'epochs': 100,
                            'data': 'atrw8.yaml',
                            'weights': 'weights/yolov5s.pt',
                            'output-s3': "s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train8"
                        }
)

#### 1/16 of Training Set

In [7]:
estimator = TensorFlow(entry_point='train.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 640,
                            'batch-size': 16,
                            'epochs': 100,
                            'data': 'atrw16.yaml',
                            'weights': 'weights/yolov5s.pt',
                            'output-s3': "s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train16"
                        }
)

## Run Training Job

In [ ]:
estimator.fit({'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets',
               'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
               'train': f's3://{bucket}/atrw/detection/train/'})

2021-05-26 10:43:56 Starting - Starting the training job...
2021-05-26 10:44:20 Starting - Launching requested ML instancesProfilerReport-1622025832: InProgress
......
2021-05-26 10:45:26 Starting - Preparing the instances for training.........
2021-05-26 10:46:41 Downloading - Downloading input data......
2021-05-26 10:47:56 Training - Training image download completed. Training in progress..2021-05-26 10:47:57.547194: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-26 10:47:57.551261: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-26 10:47:57.665767: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-26 10:48:00,107 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-05-26 1